In [92]:
from azure.cognitiveservices.language.luis.authoring import LUISAuthoringClient
from azure.cognitiveservices.language.luis.authoring.models import ApplicationCreateObject
from azure.cognitiveservices.language.luis.runtime import LUISRuntimeClient
from msrest.authentication import CognitiveServicesCredentials
from functools import reduce

import json, time, uuid

import keys

In [93]:
authoringKey = keys.authoringKey
authoringEndpoint = keys.authoringEndpoint
predictionKey = keys.predictionKey
predictionEndpoint = keys.predictionEndpoint

In [64]:
client = LUISAuthoringClient(authoringEndpoint, CognitiveServicesCredentials(authoringKey))

In [98]:
app_id = client.apps.add({
            'name': "FlyMe",
            'initial_version_id':  "0.1",
            'description': "Flight booking chatbot app",
            'culture': 'en-us',
        })

print("Created app.")

Created app.


In [66]:
luis_app = client.apps.get(app_id=app_id)

app_version = luis_app.active_version

In [67]:
entities = ['or_city', 'dst_city', 'str_date', 'end_date', 'budget']

In [68]:
# Create intent into the model
intent_name = "BookFlight"

In [70]:
# Create entities and entity features into the model
for entity in entities:
    entity_id = client.model.add_entity(app_id, app_version, name=entity)
    print(f"{entity} entity created with id {entity_id}")

or_city entity created with id 6e2767fa-8120-49f8-a471-91366393b82f
dst_city entity created with id b253bc4b-a48d-470c-bc3f-b6ae46e60772
str_date entity created with id 5f8525af-9b4b-4897-b9a4-75f3ecd540c5
end_date entity created with id 742f04b1-2777-4471-876c-1acc6f359f5b
budget entity created with id 5eca51f6-b89a-4602-b880-f481237b5322


In [69]:
intent_id = client.model.add_intent(app_id, app_version, name=intent_name)
print(f"{intent_name} intent created with id {intent_id}")

BookFlight intent created with id 5aba285a-c3ba-48f0-bb38-124d66830d10


In [71]:
with open("train.json") as f:
    train_data = json.load(f)

In [96]:
train_data[0:5]

[{'text': 'we want to go to las vegas from portland. 2 adults',
  'intentName': 'BookFlight',
  'entityLabels': [{'entityName': 'dst_city',
    'startCharIndex': 17,
    'endCharIndex': 26},
   {'entityName': 'or_city', 'startCharIndex': 32, 'endCharIndex': 40}]},
 {'text': 'hi, i need some help comparing a few different trips. can you help me with that?',
  'intentName': 'BookFlight',
  'entityLabels': []},
 {'text': 'listen up! im marrying my boo on saturday, and afterwards her and i want to have the best honeymoon 4600 can buy\nget us to kyoto will ya?',
  'intentName': 'BookFlight',
  'entityLabels': [{'entityName': 'dst_city',
    'startCharIndex': 123,
    'endCharIndex': 128},
   {'entityName': 'budget', 'startCharIndex': 100, 'endCharIndex': 104}]},
 {'text': 'well i am currently living permanently in a cubicle on the 8th floor of a building in puebla but i want to pretend i am somewhere else instead. fly me to leon please\nwhats the fanciest place i can go to for 3600',
  'int

In [73]:
# send the training data to the service in chunks of 100

for i in range(0, len(train_data), 100):
    j = i + 100
    if j > len(train_data):
        j = len(train_data)

    client.examples.batch(
                app_id,
                app_version,
                train_data[i:j]
            )

In [74]:
# Train the LUIS application

client.train.train_version(app_id, app_version)
waiting=True
while waiting:
    info = client.train.get_status(app_id, app_version)
    waiting = any(map(lambda x: 'Queued' == x.details.status or 'InProgress' == x.details.status, info))
    if waiting:
        print("Waiting 10 seconds for training to complete...")
        time.sleep(10)
    else:
        print("Training completed.")
        waiting = False

Waiting 10 seconds for training to complete...
Waiting 10 seconds for training to complete...
Training completed.


In [95]:
# publish the LUIS application

publish_result = client.apps.publish(app_id, app_version, is_staging=False, region="west-europe")
endpoint = publish_result.endpoint_url + "?subscription-key=" + predictionKey + "&q="

In [83]:
runtime_credentials = CognitiveServicesCredentials(predictionKey)
client_runtime = LUISRuntimeClient(endpoint=predictionEndpoint, credentials=runtime_credentials)

predictionRequest = {"query" : "I want a trip from new york to florida from 10/10/2019 to 10/20/2019 with a budget of 1000$"}

predictionResponse = client_runtime.prediction.get_slot_prediction(app_id=app_id, slot_name="Production", prediction_request=predictionRequest)

predictionResponse.prediction.entities

{'or_city': ['new york'], 'dst_city': ['florida'], 'budget': ['1000$']}